# An example how to check different types of group commutator rotations


#### 1. Loading VQE results

In [1]:
from boostvqe.utils import *
gci = initialize_gci_from_vqe()
print_vqe_comparison_report(gci)

[Qibo 0.2.9|INFO|2024-06-19 12:10:39]: Using numpy backend on /CPU:0
[Qibo 0.2.9|INFO|2024-06-19 12:10:41]: Using numpy backend on /CPU:0
INFO:qibo.config:Using numpy backend on /CPU:0


In the VQE numpy
VQE energy is -14.8569 and the DBQA yields -14.8569. 
The target energy is -15.27613 which means the difference is for VQE     0.41923 and of the DBQA 0.41923         which can be compared to the spectral gap 1.08767.
The relative difference is for VQE 2.74436%     and for DBQA 2.74436%.The energetic fidelity witness for the ground state for the
      VQE is 0.61456 
        and DBQA 0.61456


In [2]:
type(gci.mode_double_bracket_rotation.name)

str

In [3]:
gci.eo_d = MagneticFieldEvolutionOracle([1+np.sin(x/3) for x in range(10)])
mode_dbr, minimizer_s, eo_d = select_recursion_step_circuit(
    gci, mode_dbr_list = [DoubleBracketRotationType.group_commutator_reduced,
                          DoubleBracketRotationType.group_commutator_mix_twice,
                          DoubleBracketRotationType.group_commutator_reduced_twice,
                          DoubleBracketRotationType.group_commutator_third_order_reduced,
                          DoubleBracketRotationType.group_commutator_third_order_reduced_twice],step_grid = np.linspace(1e-5,2e-2,10),please_be_visual = True)


AttributeError: type object 'DoubleBracketRotationType' has no attribute 'group_commutator_third_order_twice'

In [ ]:
sadads

NameError: name 'sadads' is not defined

In [ ]:
print_vqe_comparison_report(gci)

VQE energy is -14.8569 and the DBQA yields -14.8569. 
The target energy is -15.27613 which means the difference is for VQE     0.41923 and of the DBQA 0.41923         which can be compared to the spectral gap 1.08767.
The relative difference is for VQE 2.74436%     and for DBQA 2.74436%.The energetic fidelity witness for the ground state for the
      VQE is 0.61456 
        and DBQA 0.61456


In [ ]:
gci = execute_selected_recursion_step( gci, mode_dbr, minimizer_s, eo_d, please_be_verbose = True )

The boosting circuit used 0 CNOT gates coming from compiled XXZ evolution and 70 CZ gates from VQE.
For 10 qubits this gives n_CNOT/n_qubits = 0.0 and n_CZ/n_qubits = 7.0
VQE energy is -14.8569 and the DBQA yields -14.8569. 
The target energy is -15.27613 which means the difference is for VQE     0.41923 and of the DBQA 0.41923         which can be compared to the spectral gap 1.08767.
The relative difference is for VQE 2.74436%     and for DBQA 2.74436%.The energetic fidelity witness for the ground state for the
      VQE is 0.61456 
        and DBQA 0.61456
The boosting circuit used 135 CNOT gates coming from compiled XXZ evolution and 490 CZ gates from VQE.
For 10 qubits this gives n_CNOT/n_qubits = 13.5 and n_CZ/n_qubits = 49.0
VQE energy is -14.8569 and the DBQA yields -13.90862. 
The target energy is -15.27613 which means the difference is for VQE     0.41923 and of the DBQA 1.36751         which can be compared to the spectral gap 1.08767.
The relative difference is for VQE 2.74

In [ ]:
gci.loss()

-13.908624919513326

In [ ]:
mode_dbr, minimizer_s, eo_d = select_recursion_step_circuit(
    gci, mode_dbr_list = [DoubleBracketRotationType.group_commutator_third_order_reduced],step_grid = np.linspace(3e-13,11e-2,10),please_be_visual = True)


KeyboardInterrupt: 

In [ ]:
execute_selected_recursion_step( gci, mode_dbr, minimizer_s, eo_d, please_be_verbose = True )


The boosting circuit used 135 CNOT gates coming from compiled XXZ evolution and 490 CZ gates from VQE.
For 10 qubits this gives n_CNOT/n_qubits = 13.5 and n_CZ/n_qubits = 49.0
VQE energy is -14.8569 and the DBQA yields -13.90862. 
The target energy is -15.27613 which means the difference is for VQE     0.41923 and of the DBQA 1.36751         which can be compared to the spectral gap 1.08767.
The relative difference is for VQE 2.74436%     and for DBQA 8.95191%.The energetic fidelity witness for the ground state for the
      VQE is 0.61456 
        and DBQA -0.25729
The boosting circuit used 765 CNOT gates coming from compiled XXZ evolution and 2450 CZ gates from VQE.
For 10 qubits this gives n_CNOT/n_qubits = 76.5 and n_CZ/n_qubits = 245.0
VQE energy is -14.8569 and the DBQA yields -13.90862. 
The target energy is -15.27613 which means the difference is for VQE     0.41923 and of the DBQA 1.36751         which can be compared to the spectral gap 1.08767.
The relative difference is for

In [ ]:
from boostvqe.models.dbi.utils_gci_optimization import *

d_params_test, s, min_loss = gradient_descent_circuits(
gci,times_choose_step=np.linspace(1e-3,1e-2,10),
 train_epochs=5,
   d_params_init=eo_d.b_list,
   lr=10
)

In [ ]:
c =Circuit(2)
c.add(gates.CNOT(0,1))
c.add(gates.RZ(1,theta = np.pi))
c.add(gates.CNOT(0,1))
u = c.unitary()

In [ ]:
u

array([[6.123234e-17-1.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17+1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17+1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17-1.j]])

In [ ]:
c =Circuit(1)
c.add(gates.RZ(0,theta = np.pi))
c.unitary()

array([[6.123234e-17-1.j, 0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17+1.j]])